## **Machine Learning - WNBA Playoffs Prediction**
This notebook will focus on the undestanding of the data. We will be using SQLite to store the data due to its scalability & the fact that it's a relational schema.

https://docs.python.org/3/library/sqlite3.html

Import sqlite3 and connect to database file

### **Imports**

In [1]:
import pandas as pd
import sqlite3
import prep_utils as pu 
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt

### **Database Connection Setup**

In [2]:
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()

[df_awards, df_coaches, df_players_teams, df_players, df_series_post, df_teams_post, df_teams] = pu.db_to_pandas(db)

***Prepare Coaches Dataframe***

In [3]:
#df_new_coaches = pu.prepare_coaches(df_coaches, df_awards,10)
#pu.group_coaches(df_new_coaches)

***Prepare Players Dataframe***


In [4]:
#df_new_players_teams = pu.prepare_player_teams(df_players_teams,df_awards,10)

***Prepare Teams Dataframe***

In [5]:
new_teams = pu.prepare_teams(df_teams,df_teams_post,3)
print(new_teams.to_string())

Dropping divID in Teams...
Dropping ldID in Teams...
Dropping seeded in Teams...
Dropping tmORB, tmDRB, tmTRB, opptmORB, opptmDRB, opptmTRB in Teams...
Dropping GP, homeW, homeL, awayW, awayL, confW, confL, min, attend, name, confID, franchID & arena in Teams...
Creating attribute winrate Teams...
Dropping won & lost in Teams...
Converting Target PLAYOFF to binary onTeams...
Creating attribute winrate Teams...
Creating attribute PlayOffs winrate Teams...
     year tmID      rank  playoff        o_fgm        o_fga       o_ftm       o_fta       o_3pm       o_3pa      o_oreb      o_dreb        o_reb      o_asts        o_pf       o_stl        o_to       o_blk        o_pts        d_fgm        d_fga       d_ftm       d_fta       d_3pm       d_3pa      d_oreb      d_dreb        d_reb      d_asts        d_pf       d_stl        d_to       d_blk        d_pts   Winrate  PO_Winrate
0       9  ATL  0.000000        0     0.000000     0.000000    0.000000    0.000000    0.000000    0.000000    0.0000

In [6]:
df_new_player_rankings = pu.prepare_players_for_ranking(df_players_teams, df_awards)
feature_importance, df_new_players = pu.feature_importance_players(df_new_player_rankings, df_players,df_teams)
print(feature_importance)



Mean Squared Error for G: 0.2684330708661418
Feature importance for G:
fg%: 0.1370479773167128
PER: 0.10170374061524728
PPM: 0.09272395347387796
ft%: 0.09017757044764921
3pt%: 0.08817942235292531
PF: 0.07782131171990443
assists: 0.06926631950142072
turnovers: 0.06474690711203115
steals: 0.057883340406621187
dRebounds: 0.05760204373370951
blocks: 0.05253122912642495
rebounds: 0.04862220479781269
oRebounds: 0.04542077879032855
dq: 0.014674680404750885
player_awards: 0.0015985202005833795

Mean Squared Error for C-F: 0.29856666666666665
Feature importance for C-F:
blocks: 0.2444671808663648
PER: 0.1213104623141328
fg%: 0.08914332641224817
ft%: 0.08620907451305536
steals: 0.07591483197969033
assists: 0.07159593577546326
PPM: 0.07064338280447001
turnovers: 0.0632610197783736
oRebounds: 0.04943182761802856
3pt%: 0.03563788597168706
dRebounds: 0.035380173320149165
PF: 0.029681170130312854
rebounds: 0.021371075063732745
dq: 0.005952653452291133
player_awards: 0.0

Mean Squared Error for C: 0.

In [7]:
rank_players_regular = pu.ranking_players(feature_importance, df_new_players)
print(rank_players_regular)


        playerID  year     rating
1074  jacksla01w     8  27.696915
204   jacksla01w     4  27.558088
829   jacksla01w     6  26.739812
687   griffyo01w     1  24.389721
1570  desouer01w    10  22.090918
...          ...   ...        ...
860   weberma01w     8  -1.701758
56    martinu01w     6  -1.992885
1030  francde01w     1  -2.315182
1180  willili01w    10  -3.039945
1531  chambco01w     8 -10.963053

[1805 rows x 3 columns]


In [8]:
rank_playoff_players = pu.ranking_playoff_players(feature_importance, df_new_players)
print(rank_playoff_players)

        playerID  year  PostRating
395    zollsh01w     9    0.000000
1393  zirkozu01w     4    0.000000
657   zellosh01w    10    2.001313
835    zarafr01w     6    1.132658
1322  zakalok01w     1    0.000000
...          ...   ...         ...
33    abrossv01w     5    1.437785
45    abrossv01w     6    0.000000
59    abrossv01w     7    0.000000
81    abrossv01w     9    2.268522
0     abrossv01w     2    0.000000

[1805 rows x 3 columns]


In [9]:
power_ratings = pu.team_power_rating(df_teams, df_new_players)

# Get the power ratings for the teams in the 1st year
power_ratings_1 = power_ratings[power_ratings['year'] == 1]

# Order the teams by power rating
power_ratings_1 = power_ratings_1.sort_values(by=['OverallPowerRating'], ascending=False)

print(power_ratings_1)

# Get the power ratings for the teams in the 2nd year
power_ratings_2 = power_ratings[power_ratings['year'] == 2]

# Order the teams by power rating
power_ratings_2 = power_ratings_2.sort_values(by=['OverallPowerRating'], ascending=False)

print(power_ratings_2)

    year tmID         C       C-F         F       F-C       F-G         G  \
15     1  WAS  0.003727  0.000000 -0.002515  0.014686  0.000000  0.224971   
9      1  ORL  0.001582  0.000000  0.000000  0.023442  0.000000  0.190671   
1      1  CLE  0.058283  0.006206  0.114873  0.000000  0.000000  0.009204   
2      1  DET  0.002890  0.007308  0.004928  0.007872  0.000000  0.199956   
14     1  UTA  0.103087  0.000000  0.002054  0.010188  0.000000  0.007013   
12     1  SAC  0.002398  0.012964  0.055826  0.010425  0.002949  0.017185   
3      1  HOU  0.004559  0.000000  0.015662  0.024087  0.009268  0.005986   
10     1  PHO  0.005819  0.000000  0.033211  0.017340  0.000000 -0.011324   
5      1  LAS  0.010317  0.006118  0.022618  0.000000  0.000000  0.005102   
0      1  CHA  0.002668  0.000000  0.003417  0.010462  0.000000  0.016814   
13     1  SEA  0.001189  0.000000  0.002523  0.013704  0.000000  0.006635   
6      1  MIA  0.004122  0.000000  0.001828  0.007994  0.003113  0.001075   

In [10]:
pu.prepare_teams(df_teams)

TypeError: prepare_teams() missing 2 required positional arguments: 'teams_post' and 'past_years'

In [ ]:
pu.best_colleges(df_players_teams,df_teams,df_players)

,college,TotalPlayoffAppearances,CollegeRank
88,Tennessee,21,1
17,Connecticut,17,2
31,Georgia,15,3
86,Stanford,12,4
48,Louisiana Tech,11,5
...,...,...,...
80,Seton Hall,1,14
25,Florida International,1,14
82,Southern Mississippi,1,14
50,Maine,1,14


### **Data Preparation**
We will preparate the data in each table, by cleaning & formatting it so that it can be easily used by the machine learning models afterwards.